In [36]:
import numpy as np
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist

import numpy as np
from scipy.spatial import cKDTree

def read_xyz(filename):
    # Read the xyz file which now contains atom types as integers (1, 2, 3)
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    atom_count = int(lines[0].strip())
    atoms = []
    for line in lines[2:2 + atom_count]:
        parts = line.strip().split()
        atom_type = int(parts[0])  # Directly read the atom type as integer
        atoms.append([atom_type, float(parts[1]), float(parts[2]), float(parts[3])])
    return np.array(atoms)

def write_xyz(atoms, filename):
    # Mapping from numbers to atom symbols for output
    number_to_symbol = {1: "Pt", 2: "C", 3: "O"}
    
    with open(filename, 'w') as file:
        file.write(f"{len(atoms)}\n")
        file.write("Atoms with added C and O\n")
        for atom in atoms:
            symbol = number_to_symbol.get(atom[0], "Unknown")  # Convert number to symbol
            file.write(f"{symbol} {atom[1]:.5f} {atom[2]:.5f} {atom[3]:.5f}\n")

def find_surface_atoms(atoms, cutoff=2.8, max_neighbors=12):
    positions = atoms[:, 1:4]
    tree = cKDTree(positions)
    surface_atoms = []
    for i, pos in enumerate(positions):
        neighbors = tree.query_ball_point(pos, cutoff)
        if len(neighbors) - 1 < max_neighbors:  # Exclude self in neighbor count
            surface_atoms.append(i)
    return surface_atoms

def add_atoms(atoms, surface_indices, center, add_distances=(2, 1.5)):
    new_atoms = list(atoms.copy())
    positions = atoms[:, 1:4]
    for idx in surface_indices:
        normal_vector = positions[idx] - center
        normal_vector /= np.linalg.norm(normal_vector)  # Normalize the vector
        
        # Add Carbon atom
        c_position = positions[idx] + normal_vector * add_distances[0]
        new_atoms.append([2, *c_position])  # '2' is the integer identifier for 'C'
        
        # Add Oxygen atom
        o_position = c_position + normal_vector * add_distances[1]
        new_atoms.append([3, *o_position])  # '3' is the integer identifier for 'O'
    return np.array(new_atoms)

# Parameters
filename = 'coc561.xyz'
output_filename = 'modified_particle_561.xyz'
neighbor_cutoff = 3.5  # Appropriate cutoff distance for Pt-Pt in Ångstroms

# Read, process, and write the new XYZ file
atoms = read_xyz(filename)
center = np.mean([atom[1:] for atom in atoms], axis=0)
surface_indices = find_surface_atoms(atoms, cutoff=neighbor_cutoff)
new_atoms = add_atoms(atoms, surface_indices, center)
write_xyz(new_atoms, output_filename)


In [31]:
atoms

array([[ 1.        , 10.24401561, 12.26897253, 16.03154602],
       [ 1.        , 12.25698371, 10.14655733, 16.12347563],
       [ 1.        , 12.07435665, 11.95003914, 13.92421743],
       ...,
       [ 1.        , 25.93006523, 23.95365504, 20.08648208],
       [ 1.        , 25.91991047, 22.06654208, 22.05667896],
       [ 1.        , 25.73661178, 25.80297549, 18.03635779]])